## Determine Optimal Portfolio Allocation Using Monte Carlo

In [86]:
import datetime as dt
import yfinance as yf
import numpy as np
import pandas as pd
from functools import reduce
from random import randint

# 10 Companies to Design Portfolio Around
tickers = ["coke", 'googl', 'hd', 'ford', 'smg', 'jnj', 'ag', "aapl", "a", "abc", "ba"]
start = "2010-01-01"
end = "2020-01-01"

# Grab yfinance Data
df_portfolio = yf.download(tickers, start, end).reset_index()

[*********************100%***********************]  11 of 11 completed


### Initial Parameters

In [87]:
iterations = 10000

In [88]:
# Calculate Log Returns
df_stock_data = df_portfolio.set_index("Date").copy(deep=True)
df_stock_data = df_stock_data["Adj Close"].dropna()

companies = df_stock_data.columns
company_returns = []

for company in companies:
    df_stock_data[f"{company}".lower()] = np.log(df_stock_data[company] / df_stock_data[company].shift(1))
    company_returns.append(f"{company}".lower())

# Drop Null Returns
df_stock_data.dropna(inplace=True)

df_returns = df_stock_data[company_returns].copy(deep=True)
df_returns

,a,aapl,abc,ag,ba,coke,ford,googl,hd,jnj,smg
Date,,,,,,,,,,,
2010-01-05,-0.010922,0.001727,-0.007160,0.012121,0.032227,-0.025514,-0.014889,-0.004413,0.007298,-0.011663,-0.006953
2010-01-06,-0.003559,-0.016034,-0.009500,0.030845,0.029883,0.004012,0.019803,-0.025532,-0.003468,0.008101,0.014112
2010-01-07,-0.001297,-0.001850,-0.016167,-0.028438,0.039684,-0.013822,-0.019803,-0.023555,0.011745,-0.007163,0.011903
2010-01-08,-0.000325,0.006626,0.010807,0.028438,-0.009693,-0.004457,-0.020203,0.013243,-0.004820,0.003432,-0.000503
2010-01-11,0.000649,-0.008861,0.011071,-0.004684,-0.011922,-0.003306,0.035091,-0.001513,-0.028703,0.000156,0.006778
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,0.000469,0.000950,-0.002105,0.062979,-0.013571,0.005742,-0.020001,-0.004601,0.006638,-0.003489,0.000845
2019-12-26,0.001757,0.019646,-0.004341,0.016584,-0.009292,0.021414,0.039609,0.013329,0.000680,-0.000686,-0.000188
2019-12-27,-0.000351,-0.000380,-0.001648,-0.022454,0.000667,0.003445,-0.039609,-0.005763,-0.003857,-0.000549,0.001500


### Calculate Mean Returns

In [89]:
# Calculating Mean Returns

df_returns_mean = pd.DataFrame([{col: ((df_returns[col].mean() * 252)) for col in df_returns.columns}])

df_returns_mean

,a,aapl,abc,ag,ba,coke,ford,googl,hd,jnj,smg
0,0.141839,0.24062,0.131374,0.109753,0.200509,0.17796,-0.071951,0.145445,0.227154,0.111679,0.129202


### Calculate Risk

In [90]:
# Calculating Covariance Matrix
df_covariance_matrix = df_returns.cov() * 252

### Run Monte Carlo

In [91]:
risk_free_rate = 0

df_final = pd.DataFrame(columns=tickers)

# Randomly Assign Allocation
for iteration in range(iterations):
    new_df = {}
    new_df_list = []

    for ticker in tickers:
        new_df[ticker] = randint(0,100)
    new_df_list.append(new_df)

    allocation = pd.DataFrame(new_df_list)
    allocation_sum = allocation.sum(axis=1)
    
    for ticker in tickers:
        allocation[ticker] = allocation[ticker].apply(lambda x: x/allocation_sum)

    expected_return = (df_returns_mean * allocation).sum().sum()
    portfolio_risk = np.sqrt(reduce(np.dot, [allocation, df_covariance_matrix, allocation.T]))

    allocation["expected_return"] = expected_return
    allocation["portfolio_risk"] = portfolio_risk
    allocation["sharp_ratio"] = (expected_return - risk_free_rate) / portfolio_risk

    df_final = pd.concat([df_final, allocation])

df_final.sort_values("sharp_ratio", ascending=False)


,coke,googl,hd,ford,smg,jnj,ag,aapl,a,abc,ba,expected_return,portfolio_risk,sharp_ratio
0,0.028017,0.015086,0.196121,0.012931,0.079741,0.019397,0.019397,0.176724,0.099138,0.155172,0.198276,0.182124,0.146576,1.242519
0,0.052846,0.109756,0.146341,0.004065,0.050813,0.016260,0.008130,0.117886,0.168699,0.126016,0.199187,0.176379,0.147751,1.193759
0,0.022222,0.166667,0.150000,0.003704,0.012963,0.042593,0.037037,0.179630,0.103704,0.103704,0.177778,0.179700,0.150642,1.192896
0,0.076733,0.029703,0.237624,0.009901,0.106436,0.049505,0.022277,0.180693,0.059406,0.180693,0.047030,0.178038,0.149650,1.189695
0,0.024283,0.064018,0.147903,0.033113,0.008830,0.086093,0.035320,0.189845,0.041943,0.192053,0.176600,0.171749,0.144490,1.188661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.108844,0.030612,0.020408,0.170068,0.061224,0.040816,0.289116,0.085034,0.146259,0.023810,0.023810,0.109529,0.287012,0.381620
0,0.009756,0.119512,0.019512,0.221951,0.014634,0.156098,0.175610,0.026829,0.068293,0.175610,0.012195,0.087837,0.230776,0.380614
0,0.045894,0.048309,0.033816,0.241546,0.024155,0.072464,0.239130,0.164251,0.055556,0.014493,0.060386,0.104368,0.274298,0.380493
0,0.044586,0.057325,0.009554,0.171975,0.133758,0.143312,0.248408,0.015924,0.095541,0.035032,0.044586,0.097544,0.261485,0.373039
